In [11]:
from random import random
from typing import Callable
import sys; sys.path.append("..")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm

from sampling.distributions import Normal, Triangle
from sampling.protocols import Sampler
from sampling.samplers import FunctionSampler, RejectionSampler, UniformSampler

In [12]:
sns.set()

In [13]:
plots_path = "../plots"

In [14]:
x = np.linspace(0, 4, 1_000)
dist = Triangle(1, 3)
y = [dist.pdf(x_) for x_ in x]

sns.lineplot(data={"x": x, "y": y}, x="x", y="y")
plt.savefig(f"{plots_path}/triangle.png")

In [15]:
sns.histplot(
    data={"x": [random() for _ in range(100_000)]},
    x="x",
)
plt.savefig(f"{plots_path}/uniform1.png")

In [16]:
def plot_pdf(sampler: Sampler, n=1_000_000):
    samples = [sampler.draw() for _ in tqdm(range(n))]
    sns.kdeplot(data={"x": samples}, x="x", bw_adjust=0.5)


plot_pdf(UniformSampler(5, 10))
plt.savefig(f"{plots_path}/uniform2.png")

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [17]:
def transform_sampler(sampler: Sampler, f: Callable[[float], float]) -> Sampler:
    return FunctionSampler(func=lambda: f(sampler.draw()))

In [18]:
squared_uniform_sampler = transform_sampler(UniformSampler(0, 1), lambda x: x**2)
plot_pdf(squared_uniform_sampler)

  0%|          | 0/1000000 [00:00<?, ?it/s]

Let $X$ be distributed uniformly between $l$ and $u$. Then $p(x) = \frac{1}{u - l}$.

We have some unknown distribution $T$. We know the pdf of $T$ is $f$.

We want to know what the pdf of $f(X)$ is.

We know that $p(f(X)=f(x)) = \frac{1}{u - l}$ for $x \in [l, u]$. 

However, we can't get from this to $p(X = x)$ unless we know how to invert $f$. For some special functions we have that, but for a general function defined computationally, we don't.

In [19]:
sampler = RejectionSampler(Normal(0, 1), lower=-20, upper=20)
plot_pdf(sampler, n=100_000)
plt.savefig(f"{plots_path}/normal_pdf.png")

  0%|          | 0/100000 [00:00<?, ?it/s]

In [10]:
sns.reset_orig()
xs = np.linspace(-20, 20, 100)
ys = Normal(0, 1).pdf(xs)
fig = sns.lineplot(data={"xs": xs, "ys": ys}, x="xs", y="ys")
fig.set_facecolor("orange")
plt.fill_between(xs, ys)
plt.savefig(f"{plots_path}/orange_and_blue.png")